In [1]:
%%bash

cd /DrakeDP
if test -f requirements.txt
    then
    sed -i '/jedi/d;/jupyter/d;' ./requirements.txt
    pip install -r ./requirements.txt "pyzmq<25.0.0"
    else echo "There's no requiremnts.txt, so nothing to install. This is the case with most projects."
fi


bash: line 2: cd: /DrakeDP: No such file or directory


There's no requiremnts.txt, so nothing to install. This is the case with most projects.


In [4]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)

In [5]:
meshcat=StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000
